In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import pickle as pkl
# import notebook_helper
import sys
from neuro.sasc.modules import fmri_module
import seaborn as sns
from neuro.sasc.data.data import TASKS_D3
import neuro.sasc.data.data
import numpy as np
from scipy.special import softmax
from collections import defaultdict

### Look at datasets

In [ ]:
d = defaultdict(list)
for task_name in TASKS_D3:
    task = TASKS_D3[task_name]
    d['Module name'].append(
        task_name[task_name.index('_')+1:].replace('_', '-'))
    d['Groundtruth keyphrase'].append(
        neuro.sasc.data.data.get_groundtruth_keyword(task_name))
    d['Dataset explanation'].append(task['groundtruth_explanation'])
    data = task['gen_func'](return_df=True)
    # d['Example'].append(data['input'][data['label'] == 1].iloc[0])
    d['Examples'].append(len(data))
    d['Unique unigrams'].append(len(set(' '.join(data['input']).split())))
    # d['Majority class fraction'].append(data['label'].mean().round(2))
df = pd.DataFrame.from_dict(d)

# display dataframe with clipping strings
with pd.option_context('display.max_colwidth', 100):
    display(df)
    df.style.hide(axis='index').to_latex(
        '../results/figs/synthetic_examples_full.tex', hrules=True)
    # df[::3].to_latex('../results/figs/synthetic_examples.tex', index=False)

### Visualize data matrix

In [ ]:
fname = '../results/mean_preds_matrix_d3___instructor.pkl'
# fname = '../results/mean_preds_matrix_d3___facebook__opt-iml-max-30b.pkl'
# mat = pkl.load(open('../results/mean_preds_matrix_d3.pkl', 'rb'))
mat = pkl.load(open(fname, 'rb'))
n = mat.shape[0]

In [ ]:
mp = mat
mp = softmax(mp, axis=0)
# divide each row by its max value
# mp = mp / (np.max(mp, axis=1).reshape(-1, 1))
# mp = mp / (np.max(mp, axis=0).T)
# mp = np.log(mp)

plt.figure(figsize=(14, 12))
# plt.title(os.path.basename(fname))
sns.heatmap(mp, cbar_kws={'label': 'Mean response of the module'})
labs = list(map(lambda x: x[3:].replace('_', '-'), list(TASKS_D3.keys())))
plt.ylabel('Task used for prompt')
plt.xlabel('Examples from this task')
plt.yticks(np.arange(len(TASKS_D3)) + 0.5, labels=labs,
           rotation='horizontal', fontsize='small')
plt.xticks(np.arange(len(TASKS_D3)) + 0.5, labels=labs,
           rotation='vertical', fontsize='small')
plt.savefig('../results/figs/mean_preds_d3.pdf', bbox_inches='tight')
plt.show()

In [ ]:
# plt.hist(m.flatten())
# plt.xscale('log')
plt.ylabel('Value in m')
plt.xlabel('Number of points (sorted in increasing order)')
plt.plot(np.sort(mp.flatten()))
plt.xlim((0, 2500))
plt.show()